In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
def build_base(inputSize, loss_function = "mse", optimizer = "adam"):
    model = Sequential()
    model.add(Dense(inputSize[0]//2, input_shape = inputSize, activation = "relu"))
    model.add(Dense(inputSize[0], activation = "relu"))
    model.compile(loss = loss_function, optimizer = optimizer, metrics = ["mse"])

    return model

def get_encoded_data(model, decoder_layers, xTrain, loss_function = "mse", optimizer = "adam"):
    decoder_layers.append(model.layers[-1])
    model.pop()
    model.compile(loss = loss_function, optimizer = optimizer, metrics = [loss_function])
    model.summary()
    print()
    xTrainEncoded = model.predict(xTrain)
    print(xTrainEncoded.shape)
    
    return xTrainEncoded, decoder_layers

def add_layer(model, layer_dimension):
    for layer in model.layers:
        layer.trainable = False
    model.add(Dense(layer_dimension, activation = "relu"))
    model.add(Dense(layer_dimension*2, activation = "relu"))
    model.compile(loss = "mse", optimizer = "adam", metrics = ["mse"])
    model.summary()
    
    return model

In [3]:
batchSize = 250
verbose = 1
epochs = 1000
ES = EarlyStopping(monitor = "mse", patience = 20)

# 1st Autoencoder

In [4]:
model = build_base((32*32,), "mse", "adam")
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
Total params: 1,050,112
Trainable params: 1,050,112
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Treinamos a primeiro autoencoder

xTrain = np.load("../data/711_input.npy").astype("float32")

model.fit(xTrain, xTrain, batch_size = batchSize, epochs = epochs, verbose = verbose, callbacks = [ES])

Epoch 1/1000
10499/10499 [==============================] - 3s 285us/step - loss: 0.0359 - mse: 0.0359
Epoch 2/1000
 4000/10499 [==========>...................] - ETA: 0s - loss: 0.0320 - mse: 0.032 - ETA: 0s - loss: 0.0319 - mse: 0.0319

D:\Programs\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mse
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


10499/10499 [==============================] - 0s 42us/step - loss: 0.0317 - mse: 0.0317
Epoch 3/1000
10499/10499 [==============================] - 0s 41us/step - loss: 0.0313 - mse: 0.0313
Epoch 4/1000
10499/10499 [==============================] - 0s 41us/step - loss: 0.0311 - mse: 0.0311
Epoch 5/1000
10499/10499 [==============================] - 0s 43us/step - loss: 0.0311 - mse: 0.0311
Epoch 6/1000
10499/10499 [==============================] - 0s 43us/step - loss: 0.0307 - mse: 0.0307
Epoch 7/1000
10499/10499 [==============================] - 0s 44us/step - loss: 0.0306 - mse: 0.0306
Epoch 8/1000
10499/10499 [==============================] - 0s 42us/step - loss: 0.0306 - mse: 0.0306
Epoch 9/1000
10499/10499 [==============================] - 0s 41us/step - loss: 0.0306 - mse: 0.0306
Epoch 10/1000
10499/10499 [==============================] - 0s 42us/step - loss: 0.0305 - mse: 0.0305: 0s - loss: 0.0306 - mse: 0.
Epoch 11/1000
10499/10499 [==============================] - 0s 4

KeyboardInterrupt: 

# 2nd Autoencoder

In [ ]:
# Retiramos a layer de decodificação, e pegamos os dados codificados para treinar o próximo autoencoder.

xTrainEncoded, decoder_layers = get_encoded_data(model, [], xTrain)

In [ ]:
# Adicionando o segundo autoencoder

model = add_layer(model, 256)

In [ ]:
# Treinando o segundo autoencoder

model.fit(xTrain, xTrainEncoded, batch_size = batchSize, epochs = epochs, verbose = verbose, callbacks = [ES])

# 3rd Autoencoder

In [ ]:
# Training 3rd autoencoder

xTrainEncoded, decoder_layers = get_encoded_data(model, decoder_layers, xTrain)

In [ ]:
model = add_layer(model, 128)

In [ ]:
model.fit(xTrain, xTrainEncoded, batch_size = batchSize, epochs = epochs, verbose = verbose, callbacks = [ES])

# 4th Autoencoder

In [ ]:
xTrainEncoded, decoder_layers = get_encoded_data(model, decoder_layers, xTrain)

In [ ]:
model = add_layer(model, 64)

In [ ]:
model.fit(xTrain, xTrainEncoded, batch_size = batchSize, epochs = epochs, verbose = verbose, callbacks = [ES])

# 5th Autoencoder

In [ ]:
xTrainEncoded, decoder_layers = get_encoded_data(model, decoder_layers, xTrain)

In [ ]:
model = add_layer(model, 32)

In [ ]:
model.fit(xTrain, xTrainEncoded, batch_size = batchSize, epochs = epochs, verbose = verbose, callbacks = [ES])

# Add decoder layers

In [ ]:
for i in range(len(decoder_layers)-1, -1, -1):
    model.add(decoder_layers[i])
    
for layer in model.layers:
    layer.trainable = False

model.compile(loss = "mse", optimizer = "adam", metrics = ["mse"])
model.summary()

# Comparing the results with the original data

In [ ]:
evaluation = model.evaluate(xTrain, xTrain)